In [14]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity

# Load the dataset into a Pandas DataFrame
df = pd.read_csv('Q1.csv')

# Preprocess the data
df['Answer'] = df['Answer'].str.lower()
df['Answer'] = df['Answer'].str.replace('[^\w\s]', '')
df = df.dropna()

# Tokenize and encode the text data
max_words = 10000
max_length = 128

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['Answer'])
X = tokenizer.texts_to_sequences(df['Answer'])
X = pad_sequences(X, maxlen=max_length)
y = df['Ratings'].values.reshape(-1, 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the neural network model (CNN)
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the neural network model
model.fit(X_train, y_train, validation_split=0.2, epochs=10)

Question = df['Question'].iloc[0]
print(f"Associated Question: {Question}\n")

input_text = "Taj Mahal is constructed in Agra. It is made up of white marble. It was built by Shah Jahan in memory of Mumtaz Mahal"
input_text = input_text.lower()
input_text = re.sub('[^\w\s]', '', input_text)
input_encoded = tokenizer.texts_to_sequences([input_text])
input_padded = pad_sequences(input_encoded, maxlen=max_length)

# Calculate the predicted rating
predicted_rating_ = model.predict(input_padded.reshape(1, max_length))[0][0]

# Calculate cosine similarity
def calculate_cosine_similarity(user_input, dataset_answers):
    similarities = cosine_similarity(user_input, dataset_answers)
    return similarities

# Find similar answers in the dataset
user_input_embedding = model.predict(input_padded)
dataset_embeddings = model.predict(X)
similarities = calculate_cosine_similarity(user_input_embedding, dataset_embeddings)

# Number of similar answers to consider
top_n = 5

# Get the indices of the top-N similar answers
similar_indices = similarities.argsort(axis=1)[:, -top_n:][:, ::-1]

# Extract ratings of similar answers
similar_ratings = df.iloc[similar_indices.ravel()]['Ratings'].values.reshape(-1, top_n)

# Calculate the weighted average of similar ratings based on cosine similarity
weighted_ratings = np.sum(similar_ratings * similarities[:, -top_n:], axis=1) / np.sum(similarities[:, -top_n:], axis=1)
predicted_rating = weighted_ratings

print(f"Predicted Rating (Weighted Average): {predicted_rating_:.2f}")

# Evaluate the model
y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Calculate R-squared (R^2)
r2 = r2_score(y_test, y_pred)
print(f"R-squared (R^2): {r2:.4f}")

<ipython-input-14-1c9e48f80058>:17: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Answer'] = df['Answer'].str.replace('[^\w\s]', '')


Epoch 1/10
22/22 [==============================] - 3s 90ms/step - loss: 14.7040 - mse: 14.7040 - val_loss: 7.7855 - val_mse: 7.7855
Epoch 2/10
22/22 [==============================] - 1s 55ms/step - loss: 4.1634 - mse: 4.1634 - val_loss: 2.4669 - val_mse: 2.4669
Epoch 3/10
22/22 [==============================] - 1s 54ms/step - loss: 1.3037 - mse: 1.3037 - val_loss: 0.6327 - val_mse: 0.6327
Epoch 4/10
22/22 [==============================] - 1s 53ms/step - loss: 0.4353 - mse: 0.4353 - val_loss: 0.2081 - val_mse: 0.2081
Epoch 5/10
22/22 [==============================] - 1s 53ms/step - loss: 0.1058 - mse: 0.1058 - val_loss: 0.0216 - val_mse: 0.0216
Epoch 6/10
22/22 [==============================] - 1s 52ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.0067 - val_mse: 0.0067
Epoch 7/10
22/22 [==============================] - 1s 52ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0041 - val_mse: 0.0041
Epoch 8/10
22/22 [==============================] - 1s 53ms/step - loss: 0.0034 -

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse:.4f}")

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Calculate R-squared (R^2)
r2 = r2_score(y_test, y_pred)
print(f"R-squared (R^2): {r2:.4f}")

Mean Squared Error (MSE): 0.0021
Mean Absolute Error (MAE): 0.0376
R-squared (R^2): 0.9997


In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Flatten
from keras.callbacks import EarlyStopping
import numpy as np
import re
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load the dataset into a Pandas DataFrame
csv_files = ['Q1.csv', 'Q2.csv', 'Q3.csv', 'Q4.csv', 'Q5.csv', 'Q6.csv']

# Create an empty list to store predicted ratings
predicted_ratings = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)

    # Preprocess the data
    df['Answer'] = df['Answer'].str.lower()
    df['Answer'] = df['Answer'].str.replace('[^\w\s]', '')
    df = df.dropna()

    # Tokenize and encode the text data
    max_words = 10000
    max_length = 128

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(df['Answer'])
    X = tokenizer.texts_to_sequences(df['Answer'])
    X = pad_sequences(X, maxlen=max_length)

    # Load the trained model
    model = Sequential()
    model.add(Embedding(max_words, 128, input_length=max_length))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(GlobalMaxPooling1D())
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Train the neural network model
    X_train, X_test, y_train, y_test = train_test_split(X, df['Ratings'].values.reshape(-1, 1), test_size=0.2, random_state=42)
    early_stopping = EarlyStopping(monitor='val_loss', patience=2)
    model.fit(X_train, y_train, validation_split=0.2, epochs=10, callbacks=[early_stopping])

    # Print the associated question
    associated_question = df['Question'].iloc[0]
    print(f"Associated Question: {associated_question}\n")

    # Ask for user input
    input_text = input(f"Enter answer for {csv_file}: ")
    input_text = input_text.lower()
    input_text = re.sub('[^\w\s]', '', input_text)
    input_encoded = tokenizer.texts_to_sequences([input_text])
    input_padded = pad_sequences(input_encoded, maxlen=max_length)

    # Use the trained model to predict the rating
    predicted_rating_ = model.predict(input_padded)[0][0]

    # Calculate cosine similarity
    def calculate_cosine_similarity(user_input, dataset_answers):
        similarities = cosine_similarity(user_input, dataset_answers)
        return similarities

    # Find similar answers in the dataset
    user_input_embedding = model.predict(input_padded)
    dataset_embeddings = model.predict(X)
    similarities = calculate_cosine_similarity(user_input_embedding, dataset_embeddings)

    # Number of similar answers to consider
    top_n = 5

    # Get the indices of the top-N similar answers
    similar_indices = similarities.argsort(axis=1)[:, -top_n:][:, ::-1]

    # Extract ratings of similar answers
    similar_ratings = df.iloc[similar_indices.ravel()]['Ratings'].values.reshape(-1, top_n)

    # Calculate the weighted average of similar ratings based on cosine similarity
    weighted_ratings = np.sum(similar_ratings * similarities[:, -top_n:], axis=1) / np.sum(similarities[:, -top_n:], axis=1)
    predicted_rating = weighted_ratings

    predicted_ratings.append(predicted_rating_)

    print(f"Predicted Rating for {csv_file}: {predicted_rating_:.2f}")

    # Evaluate the model
    y_pred = model.predict(X_test)

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error (MSE): {mse:.4f}")

    # Calculate Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Mean Absolute Error (MAE): {mae:.4f}")

    # Calculate R-squared (R^2)
    r2 = r2_score(y_test, y_pred)
    print(f"R-squared (R^2): {r2:.4f}")

# Calculate the combined rating
combined_rating = sum(predicted_ratings)
print(f"Combined Rating for all files: {combined_rating:.2f}")

Epoch 1/10


<ipython-input-12-b3f1c87013f9>:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Answer'] = df['Answer'].str.replace('[^\w\s]', '')


22/22 [==============================] - 3s 97ms/step - loss: 14.7543 - mse: 14.7543 - val_loss: 7.7208 - val_mse: 7.7208
Epoch 2/10
22/22 [==============================] - 2s 83ms/step - loss: 4.2205 - mse: 4.2205 - val_loss: 2.4828 - val_mse: 2.4828
Epoch 3/10
22/22 [==============================] - 1s 58ms/step - loss: 1.2895 - mse: 1.2895 - val_loss: 0.7440 - val_mse: 0.7440
Epoch 4/10
22/22 [==============================] - 1s 57ms/step - loss: 0.4538 - mse: 0.4538 - val_loss: 0.1964 - val_mse: 0.1964
Epoch 5/10
22/22 [==============================] - 1s 60ms/step - loss: 0.0855 - mse: 0.0855 - val_loss: 0.0257 - val_mse: 0.0257
Epoch 6/10
22/22 [==============================] - 1s 54ms/step - loss: 0.0131 - mse: 0.0131 - val_loss: 0.0068 - val_mse: 0.0068
Epoch 7/10
22/22 [==============================] - 1s 53ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.0026 - val_mse: 0.0026
Epoch 8/10
22/22 [==============================] - 1s 55ms/step - loss: 0.0032 - mse: 0.003

<ipython-input-12-b3f1c87013f9>:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Answer'] = df['Answer'].str.replace('[^\w\s]', '')


Epoch 1/10
21/21 [==============================] - 2s 62ms/step - loss: 17.1385 - mse: 17.1385 - val_loss: 6.2386 - val_mse: 6.2386
Epoch 2/10
21/21 [==============================] - 1s 57ms/step - loss: 5.4668 - mse: 5.4668 - val_loss: 4.0025 - val_mse: 4.0025
Epoch 3/10
21/21 [==============================] - 1s 56ms/step - loss: 2.8964 - mse: 2.8964 - val_loss: 1.5322 - val_mse: 1.5322
Epoch 4/10
21/21 [==============================] - 2s 86ms/step - loss: 0.8615 - mse: 0.8615 - val_loss: 0.2406 - val_mse: 0.2406
Epoch 5/10
21/21 [==============================] - 2s 78ms/step - loss: 0.1020 - mse: 0.1020 - val_loss: 0.0477 - val_mse: 0.0477
Epoch 6/10
21/21 [==============================] - 1s 56ms/step - loss: 0.0262 - mse: 0.0262 - val_loss: 0.0154 - val_mse: 0.0154
Epoch 7/10
21/21 [==============================] - 1s 56ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.0084 - val_mse: 0.0084
Epoch 8/10
21/21 [==============================] - 1s 57ms/step - loss: 0.0062 -

...

14/14 [==============================] - 2s 65ms/step - loss: 22.6258 - mse: 22.6258 - val_loss: 11.6321 - val_mse: 11.6321
Epoch 2/10
14/14 [==============================] - 1s 54ms/step - loss: 8.5598 - mse: 8.5598 - val_loss: 5.6471 - val_mse: 5.6471
Epoch 3/10
14/14 [==============================] - 1s 54ms/step - loss: 5.4429 - mse: 5.4429 - val_loss: 3.7547 - val_mse: 3.7547
Epoch 4/10
14/14 [==============================] - 1s 56ms/step - loss: 2.5209 - mse: 2.5209 - val_loss: 1.1819 - val_mse: 1.1819
Epoch 5/10
14/14 [==============================] - 1s 55ms/step - loss: 0.7994 - mse: 0.7994 - val_loss: 0.5121 - val_mse: 0.5121
Epoch 6/10
14/14 [==============================] - 1s 76ms/step - loss: 0.3351 - mse: 0.3351 - val_loss: 0.2098 - val_mse: 0.2098
Epoch 7/10
14/14 [==============================] - 1s 98ms/step - loss: 0.1386 - mse: 0.1386 - val_loss: 0.0918 - val_mse: 0.0918
Epoch 8/10
14/14 [==============================] - 1s 65ms/step - loss: 0.0653 - mse: 0.0

<ipython-input-12-b3f1c87013f9>:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Answer'] = df['Answer'].str.replace('[^\w\s]', '')


22/22 [==============================] - 2s 61ms/step - loss: 16.7140 - mse: 16.7140 - val_loss: 6.9134 - val_mse: 6.9134
Epoch 2/10
22/22 [==============================] - 1s 55ms/step - loss: 3.1727 - mse: 3.1727 - val_loss: 1.0071 - val_mse: 1.0071
Epoch 3/10
22/22 [==============================] - 2s 74ms/step - loss: 1.0507 - mse: 1.0507 - val_loss: 0.5895 - val_mse: 0.5895
Epoch 4/10
22/22 [==============================] - 2s 78ms/step - loss: 0.7081 - mse: 0.7081 - val_loss: 0.4910 - val_mse: 0.4910
Epoch 5/10
22/22 [==============================] - 1s 53ms/step - loss: 0.5998 - mse: 0.5998 - val_loss: 0.4056 - val_mse: 0.4056
Epoch 6/10
22/22 [==============================] - 1s 53ms/step - loss: 0.5612 - mse: 0.5612 - val_loss: 0.4597 - val_mse: 0.4597
Epoch 7/10
22/22 [==============================] - 1s 54ms/step - loss: 0.5183 - mse: 0.5183 - val_loss: 0.3989 - val_mse: 0.3989
Epoch 8/10
22/22 [==============================] - 1s 56ms/step - loss: 0.5079 - mse: 0.507

<ipython-input-12-b3f1c87013f9>:24: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Answer'] = df['Answer'].str.replace('[^\w\s]', '')


6/6 [==============================] - 1s 82ms/step - loss: 27.4788 - mse: 27.4788 - val_loss: 21.2778 - val_mse: 21.2778
Epoch 2/10
6/6 [==============================] - 0s 54ms/step - loss: 17.9847 - mse: 17.9847 - val_loss: 10.4484 - val_mse: 10.4484
Epoch 3/10
6/6 [==============================] - 0s 60ms/step - loss: 8.3458 - mse: 8.3458 - val_loss: 4.8329 - val_mse: 4.8329
Epoch 4/10
6/6 [==============================] - 0s 63ms/step - loss: 7.0207 - mse: 7.0207 - val_loss: 5.7992 - val_mse: 5.7992
Epoch 5/10
6/6 [==============================] - 0s 61ms/step - loss: 5.9045 - mse: 5.9045 - val_loss: 3.7185 - val_mse: 3.7185
Epoch 6/10
6/6 [==============================] - 0s 59ms/step - loss: 4.8875 - mse: 4.8875 - val_loss: 3.6162 - val_mse: 3.6162
Epoch 7/10
6/6 [==============================] - 0s 62ms/step - loss: 4.0425 - mse: 4.0425 - val_loss: 2.4897 - val_mse: 2.4897
Epoch 8/10
6/6 [==============================] - 0s 57ms/step - loss: 2.9659 - mse: 2.9659 - val_lo